In [1]:
import numpy as np
import math as ma
import random as rd
import os

#MNIST

In [2]:
def handle_with_mnist(x_file,y_file):
    x_file.read(4) # magic number
    image=int.from_bytes(x_file.read(4),byteorder='big') # number of images
    row=int.from_bytes(x_file.read(4),byteorder='big') # number of rows
    col=int.from_bytes(x_file.read(4),byteorder='big') # number of columns
    y_file.read(8) # magic number & number of images
    x=np.zeros((image,row*col),dtype='uint8')
    y=np.zeros(image,dtype='uint8')
    for i in range(image):
        for j in range(row*col):
            x[i][j]=int.from_bytes(x_file.read(1),byteorder='big') # all pixels of an image with 0~255
        y[i]=int.from_bytes(y_file.read(1),byteorder='big') # 0~9
    return x,y

In [3]:
def Mnist():
    train_x_file=open('train-images.idx3-ubyte','rb')
    train_y_file=open('train-labels.idx1-ubyte','rb')
    test_x_file=open('t10k-images.idx3-ubyte','rb')
    test_y_file=open('t10k-labels.idx1-ubyte','rb')
    train_x,train_y=handle_with_mnist(train_x_file,train_y_file)
    test_x,test_y=handle_with_mnist(test_x_file,test_y_file)
    return (train_x,train_y),(test_x,test_y)

In [4]:
(train_x,train_y),(test_x,test_y)=Mnist()

# Data prepare

In [6]:
N=60000
D=784
K=10

In [18]:
# Bining the gray level into 0 or 1
num_train=np.zeros(K)
num_one=np.zeros((K,D))
for i in range(N):
    num_train[train_y[i]]+=1
    for j in range(D):
        if train_x[i][j]<128:
            train_x[i][j]=0
        else:
            train_x[i][j]=1
            num_one[train_y[i]][j]+=1

#EM algorithm

In [16]:
# Initialization
probo=np.zeros((K,D))
num_prior=np.zeros(K)
# Use test data as prior
for i in range(10000):
    probo[test_y[i]]+=test_x[i]
    num_prior[test_y[i]]+=1
for i in range(K):
    probo[i]/=255*num_prior[i]
probo=probo*0.8+0.1
# Assume that Lambda of 0~9 is uniform
Lamb=0.1*np.ones((K,1))

In [19]:
def E(lamb,p): # Compute the weight of 0~9 for every image
    pc=1-p
    w=np.zeros((N,K))        
    for i in range(N):
        w[i]+=np.reshape(np.log(np.copy(lamb)),(10))
        for k in range(K):
            w[i][k]+=np.sum(train_x[i]*np.log(p[k])+(1-train_x[i])*np.log(pc[k]))
        w[i]-=w[i].min()
        w[i]/=w[i].sum()
    return w

In [20]:
def M(w,alpha1,alpha2): # Avoid from log 0
    lamb_new=np.sum(w,axis=0)+alpha2
    lamb_new/=lamb_new.sum()
    p=(train_x.transpose()@w+alpha1)/(np.sum(w,axis=0)+alpha1*D)
    return lamb_new.reshape((K,1)),p.transpose()

In [21]:
t=0
while 1:
    t+=1
    old_probo=np.copy(probo)
    w=E(Lamb,probo)
    Lamb,probo=M(w,10**-8,10**-8)
    m=np.copy(probo)
    for i in range(10):
        for j in range(784):
            if m[i][j]>0.5:
                m[i][j]=1
            else:
                m[i][j]=0
        print('class %d:'%(i))
        mat=np.reshape(np.copy(m[i]),(28,28))
        for j in range(28):
                for k in range(28):
                    print(int(mat[j][k]),end='')
                print()
        print()
    error=np.sum(np.abs(old_probo-probo))
    print('No. of Iteration: %d, Difference:'%(t),error)
    print()
    if error<10 or t>=10:
        break

class 0:
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000001100000000000
0000000000000111111000000000
0000000000001111111100000000
0000000000011111011110000000
0000000000111100000110000000
0000000001110000000010000000
0000000001110000000000000000
0000000011100000000000000000
0000000011000000000010000000
0000000011000000000010000000
0000000110000000000110000000
0000000110000000000110000000
0000000110000000001110000000
0000000110000000011110000000
0000000110000000111100000000
0000000111100011111000000000
0000000011111111110000000000
0000000001111111000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000

class 1:
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
000000000000000000000000

In [22]:
# complete data
for i in range(K):
    num_one[i]/=num_train[i]
    print('labeled class %d:'%(i))
    m=np.reshape(np.copy(num_one[i]),(28,28))
    for j in range(28):
        for k in range(28):
            if m[j][k]>0.5:
                print('1',end='')
            else:
                print('0',end='')
        print()
    print()

labeled class 0:
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000011110000000000
0000000000000111111100000000
0000000000001111111110000000
0000000000011111111110000000
0000000000111110001111000000
0000000001111000000111000000
0000000001110000000011100000
0000000011100000000011100000
0000000111100000000011100000
0000000111000000000011100000
0000000111000000000011000000
0000001110000000000111000000
0000001110000000001110000000
0000001110000000001110000000
0000001111000000111100000000
0000000111110011111000000000
0000000111111111110000000000
0000000011111111100000000000
0000000000111100000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000

labeled class 1:
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
00000000

In [23]:
# Output of guess
TP=np.zeros(K)
guess_y=np.zeros(K)
for i in range(N):
    for j in range(K):
        if w[i][j]==w[i].max():
            guess_y[j]+=1
            if train_y[i]==j:
                TP[j]+=1
            break
for i in range(K):
    print('Confusion Matrix %d:'%(i))
    print()
    print('                Predict number %d        Predict not number %d'%(i,i))
    print('Is number %d            %5.d                   %5.d'%(i,TP[i],num_train[i]-TP[i]))
    print('Isn\'t number %d         %5.d                   %5.d'%(i,guess_y[i]-TP[i],60000-guess_y[i]-(num_train[i]-TP[i])))
    print()
    print('Sensitivity (Successfully predict number %d):'%(i),TP[i]/num_train[i])
    print('Specificity (Successfully predict not number %d):'%(i),(60000-guess_y[i]-(num_train[i]-TP[i]))/(60000-num_train[i]))
    print()
    print('---------------------------------------------------------------')
print('Total iteration to converge:',t)
print('Total error rate:',1-TP.sum()/60000)

Confusion Matrix 0:

                Predict number 0        Predict not number 0
Is number 0             5276                     647
Isn't number 0          8988                   45089

Sensitivity (Successfully predict number 0): 0.8907648151274692
Specificity (Successfully predict not number 0): 0.8337925550603769

---------------------------------------------------------------
Confusion Matrix 1:

                Predict number 1        Predict not number 1
Is number 1             6622                     120
Isn't number 1         16424                   36834

Sensitivity (Successfully predict number 1): 0.9822011272619401
Specificity (Successfully predict not number 1): 0.6916144053475535

---------------------------------------------------------------
Confusion Matrix 2:

                Predict number 2        Predict not number 2
Is number 2             1141                    4817
Isn't number 2          1615                   52427

Sensitivity (Successfully predict numbe